<a href="https://colab.research.google.com/github/max-ostapenko/ga4_data_import/blob/main/GA4_Data_Import_from_Google_Cloud_Storage copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Some code here
#from ga4_data_import.sftp_server import create_instance, add_shh_pub_key
from ga4_data_import.shared import get_project_number

print(get_project_number('max-ostapenko'))

DEBUG = False

ModuleNotFoundError: No module named 'google'

## 1 Start with documentation
Read [the official documentation](https://support.google.com/analytics/answer/10071301) to have an idea about features and limitations of GA4 data import.


## 2 Create a Data Source in [GA4 Admin Panel](https://analytics.google.com/analytics/web/#/admin/data-import/)

0. Follow [the guide on configuring SFTP data source](https://support.google.com/analytics/answer/10071301#import)

In [ ]:
#@markdown 1. Get your SFTP server url by hitting RUN and deploying VM Instance
GCP_PROJECT_ID = 'max-ostapenko'  #@param {type: "string"}
ZONE = 'us-central1-a' #@param {type: "string"}
REGION = "-".join(ZONE.split("-")[:-1])
INSTANCE_NAME = 'sftp-server' #@param {type: "string"}
SFTP_USERNAME = 'sftp_user' #@param {type: "string"}

from google.colab import auth
auth.authenticate_user()
!gcloud config set project {GCP_PROJECT_ID}

[PROJECT_NUMBER] = !gcloud projects describe {GCP_PROJECT_ID} --format="value(projectNumber)" 

#@markdown 2. Enter the name of your Google Cloud Storage Bucket
BUCKET_NAME = 'sftp-server-ga4' #@param {type: "string"}

INSTANCE_IP = create_instance()["networkInterfaces"][0]["accessConfigs"][0]["natIP"]

# Give READ access to a bucket to default VM Service Account
!gcloud storage buckets add-iam-policy-binding gs://{BUCKET_NAME} \
    --member=serviceAccount:{PROJECT_NUMBER}-compute@developer.gserviceaccount.com \
    --role=roles/storage.objectViewer \
    --no-user-output-enabled

SFTP_URL = f"sftp://{INSTANCE_IP}/sftp/cost_data.csv"

print(f"""\nNow you can enter your SFTP server url:
    {SFTP_URL}
cost_data.csv file has be located in the root of the GCS Bucket.""")

In [ ]:
#@markdown 3. Paste and push Public Key from GA4 to SFTP server for authorisation
KEY_VALUE = "ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABgQDeDrtHfPb8TM6/BqANwQJdSmAGjygLE+2jMdIxfJZFRdcEogMLMrEARhzj9fjRQKw7RhudOpxkyvMjVRXlv6zvcbvmJhQCxk6yp3Cgq3U7ux2PYppBWSxei1R4DH2TSjy/k/7nuKDvdDaBwUC/WLOgjvQ5nKLbYp53RbWoPrv63YEDDEyuLjMww9rlJ+J9iL/fd1TxlZxvwVvPCxweWCzfQfLP6MWGvDmUMfe7eTg11rt3p/wdNUa/RxKC1HOJRWloGCw7dX5LgZYoCq8tKOS8rpHt8BoTFhKjoPsIw8h4uNsjOb9y6YSOQrQFaXj3o2l4X0ssLxkKQcaG+EDZ8PtUr/WnuPpfHCuTNN1Q6TnaF2t2zm4J7C6LchPMgiP/vO+wwloQpgg9tUVGC4FZxQhHTLmCoW+19KIuBu4nm+zgaeIWXuHfnPLE4UdFsIB3o76tAfZprrykjaWVLPc6PFVlvRx1PIAPTL7dHaajnv1rxuQdQlNU2a2dOE4KNoxnffE= Google Analytics Data Import Key"  #@param {type: "string"}
add_shh_pub_key(KEY_VALUE)

LOCAL_KEY = "ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQDIjG9uMbjXCMmJ6oLlI5rVlbm+eY2Ur42fCE542NAGwR64lEYmv2LkxPRzFnVU2mdF6+Ka4D8UyjM6D3fVkNfqQH8ckthcoMEgCNNK6tSU690I5YXAfGvCstGnljG4MwUWHkCFGXntqHPVnWjRfe0M88CusqazLy06VOr5hdfjTUXyxbyy4vx3kzBXE2oBXl0UZ33fI0/jHG3u6PULZ9kYu8jCXnNGp2b3ntgsMhitWTfj4GiSZF5wf6t4qDupgaW6OCrLJkgq9WS4OFdM4ZyHJadRff7KyHjsjtFoXzHOxhCrk4gj484HO86QyoA8jJE92o6k7OuVP9/Dl/w5n2jRtsvj0Fx2UJM3LH57XvflWWKOvfoYXl1v0Li78xnzLIgwDi282CHh31NNlbdm0Ux8AQXuQlLWd4R13sleLmdpqpcvrkCHktpvOIOvndar311CsPW3AvnHxNOKNAaJfE1UcP45AJ5TU6DrRJj9gAi+AJ/iET9W5XtKJOIk8JtHX6TwCXlSloZgPf+LOOscnYb5EQO93LCSyHQVwXuF35hV9DPFTXM+PUdhZCaPLwETjnoP8pQMYg1h8B6hCY9cXWlZQpxDyx4TUP/8vRuizX2oR5rLDG7p0P8x+IX6NtnuCQNockzl6DqXup/4o8SNVPHbC7Dv9uD9/zm9Pz7BK8TA+Q=="
add_shh_pub_key(LOCAL_KEY)

## 3 Test your SFTP server connention

1. Open [Google Cloud Shell terminal](https://shell.cloud.google.com/)

2. Authorise local SSH Key on the server
2. Connect to SFTP server
3. Show available files

In [ ]:
#@markdown Press RUN
if not os.path.isfile("/root/.ssh/id_rsa"):
    !ssh-keygen -t rsa -b 4096 -f ~/.ssh/id_rsa -N '' -q
[colab_sshpubkey] = !cat ~/.ssh/id_rsa.pub | awk '{print $1" "$2}'
add_shh_pub_key(colab_sshpubkey + " Google Colab")
!ssh-keygen -R {INSTANCE_IP}
!sftp -v {SFTP_USERNAME}@{INSTANCE_IP}